In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta
from IPython.display import clear_output
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2024/Conferences.csv
/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv
/kaggle/input/march-machine-learning-mania-2024/Cities.csv
/

In [2]:
start_data_season = {
    'M': 2003,
    'W': 2009
}
classifiers = {
    'M': RandomForestClassifier(max_depth=80, max_features='sqrt', min_samples_leaf=4,
                       n_estimators=1600, random_state=0),
    'W': RandomForestClassifier(max_depth=100, max_features=None, min_samples_leaf=4,
                       min_samples_split=5, n_estimators=2000, random_state=0)
}
columns = [
            'Season', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 
            'ASeedNum', 'ALastTournPct', 'ARegSznPct',
            'APts', 'APtsOpp', 'AFG', 'AFG3', 'AFT', 'AAst', 'ATO', 'AOR', 'ADR', 'AStl', 'ABlk', 'APF',
            'BSeedNum', 'BLastTournPct', 'BRegSznPct',
            'BPts', 'BPtsOpp', 'BFG', 'BFG3', 'BFT', 'BAst', 'BTO', 'BOR', 'BDR', 'BStl', 'BBlk', 'BPF',
            'Winner'
        ]
current_season = 2024

In [3]:
def get_teams_info(gender):
    teams_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}Teams.csv'
    teams_df = pd.read_csv(teams_file)
    display(teams_df)
    return teams_df

m_teams_info_df = get_teams_info('M')
w_teams_info_df = get_teams_info('W')

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2024
1,1102,Air Force,1985,2024
2,1103,Akron,1985,2024
3,1104,Alabama,1985,2024
4,1105,Alabama A&M,2000,2024
...,...,...,...,...
373,1474,Queens NC,2023,2024
374,1475,Southern Indiana,2023,2024
375,1476,Stonehill,2023,2024
376,1477,TX A&M Commerce,2023,2024


,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M
...,...,...
371,3474,Queens NC
372,3475,Southern Indiana
373,3476,Stonehill
374,3477,TX A&M Commerce


In [32]:
def get_seed_number(seed):
    if 'a' in seed or 'b' in seed:
        return 17
    return int(seed[1:])

def generate_numeric_cols(df, team):
    df[f'{team}FG'] = (df[f'{team}FGM'] * 100) / df[f'{team}FGA']
    df[f'{team}FG3'] = (df[f'{team}FGM3'] * 100) / df[f'{team}FGA3']
    df[f'{team}FT'] = (df[f'{team}FTM'] * 100) / df[f'{team}FTA']

def get_files_by_competition(gender):
    # Getting teams df
    teams_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}TeamConferences.csv'
    teams_df = pd.read_csv(teams_file)
    display(teams_df)
    
    # Getting regular season games df
    reg_szn_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}RegularSeasonDetailedResults.csv'
    reg_szn_df = pd.read_csv(reg_szn_file)
    generate_numeric_cols(reg_szn_df, 'W')
    generate_numeric_cols(reg_szn_df, 'L')
    display(reg_szn_df)
    
    # Getting tournament season games df
    tourn_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}NCAATourneyCompactResults.csv'
    tourn_df = pd.read_csv(tourn_file)
    display(tourn_df)
    
    # Getting seeding df
    seed_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}NCAATourneySeeds.csv'
    seed_df = pd.read_csv(seed_file)
    seed_df['SeedNum'] = seed_df.Seed.apply(lambda x: get_seed_number(x))
    display(seed_df)
    
    # Getting slots df
    slots_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}NCAATourneySlots.csv'
    slots_df = pd.read_csv(slots_file)
    display(slots_df)
    
    return teams_df, reg_szn_df, tourn_df, seed_df, slots_df

In [16]:
def get_reg_szn_stats(season, team, reg_szn_df):
    w_games = reg_szn_df[reg_szn_df['WTeamID'] == team].rename(columns = {
        'WFG': 'FG', 'WFG3': 'FG3', 'WFT': 'FT', 'WAst': 'Ast', 'WTO': 'TO', 'WOR': 'OR', 'WDR': 'DR', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',
        'WScore': 'Score',
        
        'LFG': 'OppFG', 'LFG3': 'OppFG3', 'LFT': 'OppFT', 'LAst': 'OppAst', 'LTO': 'OppTO', 'LOR': 'OppOR', 'LDR': 'OppDR', 'LStl': 'OppStl', 'LBlk': 'OppBlk', 'LPF': 'OppPF',
        'LScore': 'OppScore',
        })
    w_games['Won'] = 1
    
    l_games = reg_szn_df[reg_szn_df['LTeamID'] == team].rename(columns = {
        'LFG': 'FG', 'LFG3': 'FG3', 'LFT': 'FT', 'LAst': 'Ast', 'LTO': 'TO', 'LOR': 'OR', 'LDR': 'DR', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF',
        'LScore': 'Score',
        
        'WFG': 'OppFG', 'WFG3': 'OppFG3', 'WFT': 'OppFT', 'WAst': 'OppAst', 'WTO': 'OppTO', 'WOR': 'OppOR', 'WDR': 'OppDR', 'WStl': 'OppStl', 'WBlk': 'OppBlk', 'WPF': 'OppPF',
        'WScore': 'OppScore',
        })
    l_games['Won'] = 0
    
    games = pd.concat([w_games, l_games], axis=0, ignore_index=True)
    
    reg_szn_pct = (len(w_games) * 100) / len(games)
    
    return [   
               reg_szn_pct, 
               games['Score'].mean(), games['OppScore'].mean(), games['FG'].mean(), games['FG3'].mean(), games['FT'].mean(), games['Ast'].mean(), games['TO'].mean(), 
               games['OR'].mean(), games['DR'].mean(), games['Stl'].mean(), games['Blk'].mean(), games['PF'].mean()
           ]

In [17]:
def get_last_tourn_pct(season, team, tourn_df):
    wins = len(tourn_df[(tourn_df['Season'] == season-1) & (tourn_df['WTeamID'] == team)])
    total_games = len(tourn_df[(tourn_df['Season'] == season-1) & ((tourn_df['WTeamID'] == team) | (tourn_df['LTeamID'] == team))])
    
    if not total_games: return 0
    return (wins * 100) / total_games

In [18]:
def set_team_name(team_id, teams_df):
    return teams_df[teams_df['TeamID'] == team_id].reset_index().loc[0, 'TeamName']

def get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df):
    a_reg_szn_stats = get_reg_szn_stats(season, team_a, reg_szn_df)
    b_reg_szn_stats = get_reg_szn_stats(season, team_b, reg_szn_df)

    try:
        a_seed = seed_df[(seed_df['Season'] == season) & (seed_df['TeamID'] == team_a)].reset_index().loc[0, 'SeedNum']
    except:
        a_seed = 24
    try:
        b_seed = seed_df[(seed_df['Season'] == season) & (seed_df['TeamID'] == team_b)].reset_index().loc[0, 'SeedNum']
    except:
        b_seed = 24

    a_last_tourn_pct = get_last_tourn_pct(season, team_a, tourn_df)
    b_last_tourn_pct = get_last_tourn_pct(season, team_b, tourn_df)

    stats_a = [a_seed, a_last_tourn_pct] + (a_reg_szn_stats)
    stats_b = [b_seed, b_last_tourn_pct] + (b_reg_szn_stats)
    
    teams_info_df = m_teams_info_df if gender == 'M' else w_teams_info_df
    team_a_name = set_team_name(team_a, teams_info_df)
    team_b_name = set_team_name(team_b, teams_info_df)
    
    return [season, team_a, team_b, team_a_name, team_b_name] + stats_a + stats_b

def build_dataset(gender, start_data_season, reg_szn_df, tourn_df, seed_df, end_season):
    data = []

    for season in range(start_data_season + 1, end_season + 1):
        tourney_games = tourn_df[tourn_df['Season'] == season].reset_index(drop=True)
        for idx, g in tourney_games.iterrows():

            clear_output(wait=True)
            print(f"{season}: {idx}/{len(tourney_games)}")

            team_a = min([g['WTeamID'], g['LTeamID']])
            team_b = max([g['WTeamID'], g['LTeamID']])

            if team_a == g['WTeamID']:
                winner = 'A'
                team_a_score = g['WScore']
                team_b_score = g['LScore']
            else:
                winner = 'B'
                team_a_score = g['LScore']
                team_b_score = g['WScore']

            print(f"{team_a} x {team_b}")

            game_stats = get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df)

            data.append(game_stats + [winner])

    data_df = pd.DataFrame(data, columns=columns)
    display(data_df)
    
    return data_df

In [19]:
def train_model(x_train, y_train, gender):
    classifier = classifiers[gender]
    classifier.fit(x_train, y_train)
    return classifier

def predict_seasons(gender, start_data_season, data_df, detailed_results):

    acc_sum = 0
    seasons_count = 0

    for season in range(start_data_season + 2, current_season + 1):
        data_train = data_df[(data_df['Season'] < season)].reset_index(drop=True)
        data_test = data_df[(data_df['Season'] == season)].reset_index(drop=True)

        if not len(data_test):
            continue

        x_train = data_train.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
        y_train = data_train.loc[:, 'Winner']
            
        classifier = train_model(x_train, y_train, gender)

        x_test = data_test.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
        y_test = data_test.loc[:, 'Winner']

        predictions = classifier.predict(x_test)
        
        acc = accuracy_score(y_test, predictions)
        acc_sum += acc
        seasons_count += 1
        
        if detailed_results:
            print(f'\nResults for {gender} season {season}:')
            print('Accuracy predictions:', acc)

            cm = confusion_matrix(y_test, predictions)
            print('Confusion matrix:')
            cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classifier.classes_)
            cm_disp.plot()
            plt.show()

    print(f'\n\n{gender} Mean Accuracy: {acc_sum/seasons_count}')
    
    return classifier

In [20]:
def get_current_predictions(gender, classifier, reg_szn_df, tourn_df, seed_df):
    data = []
    teams = seed_df[seed_df['Season'] == current_season].TeamID.unique()
    teams.sort()
    
    for idx, team_a in enumerate(teams):
        for idx_b in range(idx + 1, len(teams)):
            team_b = teams[idx_b]
            
            clear_output(wait=True)
            print(f"{idx}/{len(teams)}")
            print(f"{team_a} x {team_b}")
            
            game_stats = get_game_stats(gender, current_season, team_a, team_b, reg_szn_df, seed_df, tourn_df)
            data.append(game_stats)
            
    data_df = pd.DataFrame(data, columns=columns[:-1])
    data_to_predict = data_df.drop(['TeamNameA', 'TeamNameB'], axis=1)
    
    probs = classifier.predict_proba(data_to_predict)
    pred = classifier.predict(data_to_predict)

    data_df['AProb'] = probs[:, 0]
    data_df['BProb'] = probs[:, 1]
    data_df['Pred'] = pred
    
    return data_df

In [ ]:
def pipeline(gender, detailed_results=False):
    teams_df, reg_szn_df, tourn_df, seed_df, slots_df = get_files_by_competition(gender)
    data_df = build_dataset(gender, start_data_season[gender], reg_szn_df, tourn_df, seed_df, current_season)
    classifier = predict_seasons(gender, start_data_season[gender], data_df, detailed_results)
    current_szn_df = get_current_predictions(gender, classifier, reg_szn_df, tourn_df, seed_df)
    display(current_szn_df)
    return current_szn_df, teams_df, slots_df

In [34]:
m_current_szn_df, m_teams_df, m_slots = pipeline('M')

66/68
1458 x 1463


,Season,TeamA,TeamB,TeamNameA,TeamNameB,ASeedNum,ALastTournPct,ARegSznPct,APts,APtsOpp,...,BAst,BTO,BOR,BDR,BStl,BBlk,BPF,AProb,BProb,Pred
0,2024,1103,1104,Akron,Alabama,14,0.0,65.198238,71.906021,67.004405,...,12.762663,13.353111,11.494935,25.085384,6.732272,4.494935,17.386397,0.273667,0.726333,B
1,2024,1103,1112,Akron,Arizona,14,0.0,65.198238,71.906021,67.004405,...,15.508499,12.701133,11.202550,25.516997,6.433428,3.696884,16.539660,0.081314,0.918686,B
2,2024,1103,1120,Akron,Auburn,14,0.0,65.198238,71.906021,67.004405,...,13.609078,13.525622,11.456808,23.537335,7.547584,4.401171,18.734993,0.324682,0.675318,B
3,2024,1103,1124,Akron,Baylor,14,0.0,65.198238,71.906021,67.004405,...,14.027356,13.205167,11.989362,23.346505,6.946809,4.033435,17.372340,0.132743,0.867257,B
4,2024,1103,1129,Akron,Boise St,14,0.0,65.198238,71.906021,67.004405,...,13.064759,12.715361,9.915663,24.209337,6.117470,2.572289,17.641566,0.420034,0.579966,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2024,1447,1458,Wagner,Wisconsin,17,0.0,52.208202,68.156151,68.526814,...,12.669516,9.732194,9.858974,23.629630,5.564103,3.170940,14.914530,0.230160,0.769840,B
2274,2024,1447,1463,Wagner,Yale,17,0.0,52.208202,68.156151,68.526814,...,13.723842,13.727273,10.018868,24.732419,6.250429,3.509434,17.926244,0.271430,0.728570,B
2275,2024,1450,1458,Washington St,Wisconsin,7,0.0,48.214286,67.712798,67.355655,...,12.669516,9.732194,9.858974,23.629630,5.564103,3.170940,14.914530,0.355824,0.644176,B
2276,2024,1450,1463,Washington St,Yale,7,0.0,48.214286,67.712798,67.355655,...,13.723842,13.727273,10.018868,24.732419,6.250429,3.509434,17.926244,0.542109,0.457891,A


In [ ]:
w_current_szn_df, w_teams_df, w_slots = pipeline('W')

21/68
3238 x 3349


In [ ]:
def build_slots(slots):
    slots = slots[slots['Season'] == current_season]
    slots = slots[slots['Slot'].str.contains('R')] 

    return slots

current_slots_m = build_slots(m_slots)
current_slots_w = build_slots(w_slots)

display(slots_m)
display(slots_w)

In [ ]:
def set_game_id(row):
    return f"{row['Season']}_{row['TeamA']}_{row['TeamB']}"

def set_impossible_games_probs(gender, current_szn_df, teams_df):
    teams_a = current_szn_df['TeamA'].unique().tolist()
    teams_b = current_szn_df['TeamB'].unique().tolist()
    tourney_teams = set(teams_a + teams_b)

    teams = teams_df[teams_df['Season'] == current_season]['TeamID'].unique().tolist()

    data = []

    for idx, team_a in enumerate(teams):
        for idx_b in range(idx + 1, len(teams)):
            team_b = teams[idx_b]

            if team_a in tourney_teams and team_b in tourney_teams:
                continue
                
            teams_info_df = m_teams_info_df if gender == 'M' else w_teams_info_df
            # There is no reason for setting team names or probabilities if these matchups will never happen in the tournament
            team_a_name = None # set_team_name(team_a, teams_info_df)
            team_b_name = None # set_team_name(team_b, teams_info_df)

            game_id = f"{current_season}_{team_a}_{team_b}"
            data.append([game_id, team_a, team_b, team_a_name, team_b_name, 0.5])

    data_df = pd.DataFrame(data, columns=['ID', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 'Pred'])
    return data_df

current_szn_df = pd.concat([m_current_szn_df, w_current_szn_df], axis=0).reset_index(drop=True)
current_szn_df['ID'] = current_szn_df.apply(lambda row: set_game_id(row), axis=1)

m_impossible_df = set_impossible_games_probs('M', m_current_szn_df, m_teams_df)
w_impossible_df = set_impossible_games_probs('W', w_current_szn_df, w_teams_df)

final_df = current_szn_df.loc[:, ['ID', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 'AProb']].rename({'AProb': 'Pred'}, axis=1)
display(final_df)
final_df = pd.concat([final_df, m_impossible_df, w_impossible_df], axis=0).sort_values(by='ID').reset_index(drop=True)

display(final_df)

In [ ]:
def create_submission_file(df):
    df_filtered = df.loc[:, ['ID', 'Pred']]
    df_filtered.to_csv('submission.csv', index=False)
    
create_submission_file(final_df)

In [ ]:
def get_rf_best_parameters(gender):
    teams_df, reg_szn_df, tourn_df, seed_df = get_files_by_competition(gender)

    data_df = build_dataset(gender, start_data_season[gender], reg_szn_df, tourn_df, seed_df, end_season = 2022) 
    x = data_df.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
    y = data_df.loc[:, 'Winner']

    from sklearn.model_selection import RandomizedSearchCV
    rs_optimizer = RandomForestClassifier()

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['sqrt', 'log2', None]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                    'max_features': max_features,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'bootstrap': bootstrap}

    rf_random = RandomizedSearchCV(estimator = rs_optimizer, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)

    rf_random.fit(x, y)

    best_random = rf_random.best_estimator_
    best_parameters = rf_random.cv_results_
    #     print(best_parameters)
    print(best_random)
    
# get_rf_best_parameters('M')